
## Train Data EDA
* 데이터 기존 정보 (샘플 수, 클래스 분포, 이미지 크기 등)
* 시각화 (클래스별 이미지 예시, 이상치 탐지 등)
* 인사이트 정리

In [3]:
# EDA를 위한 추가 라이브러리 import
import matplotlib.pyplot as plt         # 시각화 (라인 그래프, 막대 그래프 등)
import seaborn as sns                   # 고급 시각화 (히트맵, 박스플롯 등)
from collections import Counter         # 클래스 분포 계산
import cv2                              # openCV, 이미지 처리 라이브러리
from pathlib import Path                # 운영체제 독립적인 경로 관리
import matplotlib.font_manager as fm    # 폰트 관리
import pandas as pd                     # 데이터 처리
import os                               # 파일 경로 처리
import numpy as np                      # 수치 연산

# 데이터셋 경로
data_path = "/home/dev/computervisioncompetition-cv3/data"
font_path = "/home/dev/computervisioncompetition-cv3/workspaces/jaehong/notebooks/Fonts/NanumGothic-Regular.ttf"


# 시각화 설정
plt.style.use('default')                # Matplotlib 기본 스타일로 초기화
sns.set_palette("husl")                 # seaborn 팔레트 설정 (husl: 다양한 색상 팔레트)
plt.rcParams['figure.figsize'] = (12, 8)# 그래프 전체 크기 기본값 설정 (가로 12, 세로 8)
plt.rcParams['font.size'] = 10          # 글꼴 크기 기본값 설정 (축, 제목, 레이블 등 적용)

print("EDA 라이브럴 로드 완료")

In [4]:
# 데이터셋 분석을 위한 클래스
class DatasetAnalyzer:
    """
    데이터셋 분석을 위한 클래스
    """
    
    def __init__(self, train_csv_path: str, train_img_path: str, test_csv_path: str, test_img_path: str):
        """
        데이터셋 분석기 초기화
        
        Args:
            train_csv_path: 훈련 데이터 CSV 파일 경로
            train_img_path: 훈련 이미지 디렉토리 경로
            test_csv_path: 테스트 데이터 CSV 파일 경로
            test_img_path: 테스트 이미지 디렉토리 경로 
        """
        self.train_csv_path = train_csv_path
        self.train_img_path = train_img_path
        self.test_csv_path = test_csv_path
        self.test_img_path = test_img_path
        
        # 데이터 로드
        self.train_df = pd.read_csv(train_csv_path)
        self.test_df = pd.read_csv(test_csv_path)
        
    def get_basic_info(self) -> dict:
        """
        데이터셋의 기본 정보를 반환
        
        Returns:
            dict: 데이터셋 기본 정보 딕셔너리
        """
        info = {
            "train_samples": len(self.train_df),
            "test_samples": len(self.test_df),
            "num_classes": self.train_df['target'].nunique(),
            "class_names": sorted(self.train_df['target'].unique()),
            "train_img_path": self.train_img_path,
            "test_img_path": self.test_img_path
        }
        return info
    
    def analyze_class_distribution(self) -> pd.DataFrame:
        """
        클래스별 분포 분석
        
        Returns:
            pd.DataFrame: 클래스별 분포 데이터프레임
        """
        class_counts = self.train_df['target'].value_counts()
        class_distribution = pd.DataFrame({
            'class': class_counts.index,
            'count': class_counts.values,
            'percentage': (class_counts.values / len(self.train_df) * 100).round(2)
        })
        return class_distribution
    
    def analyze_image_properties(self, sample_size: int = 100) -> dict:
        """
        이미지 속성 분석 (크기, 채널 등)
        
        Args:
            sample_size: 분석할 샘플 이미지 수
            
        returns:
            dics: 이미지 속성 정보
        """
        
        # 훈련 이미지 샘플링
        sample_files = self.train_df.sample(min(sample_size, len(self.train_df)))['ID'].tolist()
        
        heights, widths, channels = [], [], []
        
        for img_file in sample_files:
            img_path = os.path.join(self.train_img_path, img_file)
            if os.path.exists(img_path):
                img = cv2.imread(img_path)
                if img is not None:
                    h, w, c = img.shape
                    heights.append(h)
                    widths.append(w)
                    channels.append(c)
        
        properties = {
            'height_stats': {
                'min': min(heights),
                'max': max(heights),
                'mean': np.mean(heights),
                'std': np.std(heights)
            },
            'width_stats': {
                'min': min(widths),
                'max': max(widths),
                'mean': np.mean(widths),
                'std': np.std(widths)
            },
            'channels': list(set(channels)),
            'sample_size': len(heights)
        }
                    
        return properties

# 데이터셋 분석기 인스턴스 생성
analyzer = DatasetAnalyzer(
    train_csv_path=data_path + "/train.csv",
    train_img_path=data_path + "/train/",
    test_csv_path=data_path + "/sample_submission.csv",
    test_img_path=data_path + "/test/"
)            

print("데이터셋 분석기 초기화 완료")

In [6]:
# 이미지 시각화를 위한 클래스
class ImageVisualizer:
    """
    이미지 시각화를 위한 클래스
    """
    
    def __init__(self, analyzer: DatasetAnalyzer):
        """
        이미지 시각화기 초기화
        
        Args:
            analyzer: 데이터셋 분석기 인스턴스
        """
        self.analyzer = analyzer
    
    def visualize_class_samples(self, samples_per_class: int = 4, figsize: tuple = (20, 15)):
        """
        각 클래스별 샘플 이미지를 시각화
        
        Args:
            samples_per_class: 클래스당 표시할 샘플 수
            figsize: 그림 크기
        """
        num_classes = len(self.analyzer.train_df['target'].unique())
        cols = samples_per_class
        rows = num_classes
        
        fig, axes = plt.subplots(rows, cols, figsize=figsize)
        fig.suptitle('Example Images per Class', fontsize=16, fontweight='bold')
        
        # 각 클래스별로 샘플 이미지 선택
        for class_idx, class_label in enumerate(sorted(self.analyzer.train_df['target'].unique())):
            class_samples = self.analyzer.train_df[
                self.analyzer.train_df['target'] == class_label
            ].sample(min(samples_per_class, len(self.analyzer.train_df[
                self.analyzer.train_df['target'] == class_label
            ])))['ID'].tolist()
            
            for sample_idx, img_file in enumerate(class_samples):
                img_path = os.path.join(self.analyzer.train_img_path, img_file)
                
                if os.path.exists(img_path):
                    img = cv2.imread(img_path)
                    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    
                    axes[class_idx, sample_idx].imshow(img_rgb)
                    axes[class_idx, sample_idx].set_title(f'Class {class_label}', fontweight='bold')
                    axes[class_idx, sample_idx].axis('off')
                else:
                    axes[class_idx, sample_idx].text(0.5, 0.5, 'Image Not Found', 
                                                   ha='center', va='center', transform=axes[class_idx, sample_idx].transAxes)
                    axes[class_idx, sample_idx].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    def analyze_image_statistics(self, sample_size: int = 100) -> dict:
        """
        이미지 통계 정보 분석 (RGB 채널별 평균, 표준편차 등)
        
        Args:
            sample_size: 분석할 샘플 이미지 수
            
        Returns:
            dict: 이미지 통계 정보
        """
        sample_files = self.analyzer.train_df.sample(min(sample_size, len(self.analyzer.train_df)))['ID'].tolist()
        
        r_values, g_values, b_values = [], [], []
        
        for img_file in sample_files:
            img_path = os.path.join(self.analyzer.train_img_path, img_file)
            if os.path.exists(img_path):
                img = cv2.imread(img_path)
                if img is not None:
                    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    # 이미지 크기 조정 (메모리 절약)
                    img_resized = cv2.resize(img_rgb, (64, 64))
                    
                    r_values.extend(img_resized[:, :, 0].flatten())
                    g_values.extend(img_resized[:, :, 1].flatten())
                    b_values.extend(img_resized[:, :, 2].flatten())
        
        statistics = {
            'r_channel': {
                'mean': np.mean(r_values),
                'std': np.std(r_values),
                'min': np.min(r_values),
                'max': np.max(r_values)
            },
            'g_channel': {
                'mean': np.mean(g_values),
                'std': np.std(g_values),
                'min': np.min(g_values),
                'max': np.max(g_values)
            },
            'b_channel': {
                'mean': np.mean(b_values),
                'std': np.std(b_values),
                'min': np.min(b_values),
                'max': np.max(b_values)
            },
            'sample_size': len(sample_files)
        }
        
        return statistics

# 이미지 시각화기 인스턴스 생성
visualizer = ImageVisualizer(analyzer)

print(" 이미지 시각화기 초기화 완료")

In [7]:

# 데이터셋 기본 정보 출력
print("=" * 60)
print(" 데이터셋 기본 정보")
print("=" * 60)

basic_info = analyzer.get_basic_info()
for key, value in basic_info.items():
    print(f"{key}: {value}")

print("\n" + "=" * 60)
print(" 클래스별 분포 분석")
print("=" * 60)

class_distribution = analyzer.analyze_class_distribution()
print(class_distribution.to_string(index=False))

# 클래스 분포 시각화
plt.figure(figsize=(15, 6))

# 한글 폰트 설정
fontprop = fm.FontProperties(fname=font_path)

# 서브플롯 1: 클래스별 개수
plt.subplot(1, 2, 1)
plt.bar(class_distribution['class'], class_distribution['count'], 
        color=plt.cm.Set3(np.linspace(0, 1, len(class_distribution))))
plt.title('클래스별 샘플 수', fontsize=14, fontweight='bold', fontproperties=fontprop)
plt.xlabel('클래스', fontproperties=fontprop)
plt.ylabel('샘플 수', fontproperties=fontprop)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 서브플롯 2: 클래스별 비율
plt.subplot(1, 2, 2)
plt.pie(class_distribution['count'], 
        labels=class_distribution['class'], 
        autopct='%1.1f%%',
        colors=plt.cm.Set3(np.linspace(0, 1, len(class_distribution))))
plt.title('클래스별 분포 비율', fontsize=14, fontweight='bold', fontproperties=fontprop)

plt.tight_layout()
plt.show()

# 클래스 분포 통계
print(f"\n 클래스 분포 통계:")
print(f"최대 클래스 샘플 수: {class_distribution['count'].max()}")
print(f"최소 클래스 샘플 수: {class_distribution['count'].min()}")
print(f"평균 클래스 샘플 수: {class_distribution['count'].mean():.1f}")
print(f"클래스 불균형 비율: {class_distribution['count'].max() / class_distribution['count'].min():.2f}")


In [8]:
# 클래스별 샘플 이미지 시각화
print("=" * 60)
print(" 클래스별 샘플 이미지 시각화")
print("=" * 60)

# 클래스별 샘플 이미지 표시
visualizer.visualize_class_samples(samples_per_class=3, figsize=(60,80))

In [9]:
# 이미지 속성 분석
print("=" * 60)
print(" 이미지 속성 분석")
print("=" * 60)

image_properties = analyzer.analyze_image_properties(sample_size=len(analyzer.train_df))

print(f"분석 샘플 개수: {image_properties['sample_size']}")
print(f"채널 수: {image_properties['channels']}")

print("\n 이미지 크기 통계:")
print(f"세로(Height) - 최소: {image_properties['height_stats']['min']}, "
      f"최대: {image_properties['height_stats']['max']}, "
      f"평균: {image_properties['height_stats']['mean']:.1f}")

print(f"가로(Width) - 최소: {image_properties['width_stats']['min']}, "
      f"최대: {image_properties['width_stats']['max']}, "
      f"평균: {image_properties['width_stats']['mean']:.1f}")

# 실제 이미지 크기 분석을 위한 샘플 이미지들
sample_files = analyzer.train_df.sample(min(50, len(analyzer.train_df)))['ID'].tolist()
heights, widths = [], []

for img_file in sample_files:
    img_path = os.path.join(analyzer.train_img_path, img_file)
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        if img is not None:
            h, w, _ = img.shape
            heights.append(h)
            widths.append(w)       
                 
def create_image_size_plots(heights, widths):
    """
    이미지 크기 분포 시각화 함수
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 높이 분포 히스토그램
    axes[0, 0].hist(heights, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].set_title('Image Height Distribution', fontweight='bold')
    axes[0, 0].set_xlabel('Height (pixels)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 너비 분포 히스토그램
    axes[0, 1].hist(widths, bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[0, 1].set_title('Image Width Distribution', fontweight='bold')
    axes[0, 1].set_xlabel('Width (pixels)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 높이 vs 너비 스캐터 플롯
    axes[1, 0].scatter(widths, heights, alpha=0.6, color='green')
    axes[1, 0].set_title('Image Size Distribution (Width vs Height)', fontweight='bold')
    axes[1, 0].set_xlabel('Width (pixels)')
    axes[1, 0].set_ylabel('Height (pixels)')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 종횡비 분포
    aspect_ratios = [w/h for w, h in zip(widths, heights)]
    axes[1, 1].hist(aspect_ratios, bins=20, alpha=0.7, color='gold', edgecolor='black')
    axes[1, 1].set_title('Image Aspect Ratio Distribution', fontweight='bold')
    axes[1, 1].set_xlabel('Aspect Ratio (Width/Height)')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return aspect_ratios
# 이미지 크기 분포 시각화
aspect_ratios = create_image_size_plots(heights, widths)

print(f"\n 이미지 사이즈 통계:")
print(f"평균 비율: {np.mean(aspect_ratios):.2f}")
print(f"비율 표준편차: {np.std(aspect_ratios):.2f}")
print(f"정사각형 이미지 비율: {sum(1 for ratio in aspect_ratios if 0.9 <= ratio <= 1.1) / len(aspect_ratios) * 100:.1f}%")


In [11]:
# 이미지 통계 정보 분석
print("=" * 60)
print(" 이미지 통계 정보 분석")
print("=" * 60)

# RGB 채널별 통계 분석
image_stats = visualizer.analyze_image_statistics(sample_size=len(analyzer.train_df))

print(f"분석 샘플 개수: {image_stats['sample_size']}")
print(f"\n🔴 Red 채널 통계:")
print(f"  평균: {image_stats['r_channel']['mean']:.2f}")
print(f"  표준편차: {image_stats['r_channel']['std']:.2f}")
print(f"  최소: {image_stats['r_channel']['min']}")
print(f"  최대: {image_stats['r_channel']['max']}")

print(f"\n🟢 Green 채널 통계:")
print(f"  평균: {image_stats['g_channel']['mean']:.2f}")
print(f"  표준편차: {image_stats['g_channel']['std']:.2f}")
print(f"  최소: {image_stats['g_channel']['min']}")
print(f"  Max: {image_stats['g_channel']['max']}")

print(f"\n🔵 Blue 채널 통계:")
print(f"  평균: {image_stats['b_channel']['mean']:.2f}")
print(f"  표준편차: {image_stats['b_channel']['std']:.2f}")
print(f"  최소: {image_stats['b_channel']['min']}")
print(f"  최대: {image_stats['b_channel']['max']}")

# 실제 픽셀 값 분석을 위한 샘플 이미지들
sample_files = analyzer.train_df.sample(min(50, len(analyzer.train_df)))['ID'].tolist()
r_values, g_values, b_values = [], [], []

for img_file in sample_files:
    img_path = os.path.join(analyzer.train_img_path, img_file)
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        if img is not None:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_resized = cv2.resize(img_rgb, (32, 32))  # 메모리 절약을 위해 작게
            
            r_values.extend(img_resized[:, :, 0].flatten())
            g_values.extend(img_resized[:, :, 1].flatten())
            b_values.extend(img_resized[:, :, 2].flatten())
            
def create_rgb_channel_plots(r_values, g_values, b_values):
    """
    RGB 채널 분포 시각화 함수 (영어 제목 사용)
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Red 채널 히스토그램
    axes[0, 0].hist(r_values, bins=50, alpha=0.7, color='red', edgecolor='black')
    axes[0, 0].set_title('Red Channel Distribution', fontweight='bold')
    axes[0, 0].set_xlabel('Pixel Value')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Green 채널 히스토그램
    axes[0, 1].hist(g_values, bins=50, alpha=0.7, color='green', edgecolor='black')
    axes[0, 1].set_title('Green Channel Distribution', fontweight='bold')
    axes[0, 1].set_xlabel('Pixel Value')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Blue 채널 히스토그램
    axes[1, 0].hist(b_values, bins=50, alpha=0.7, color='blue', edgecolor='black')
    axes[1, 0].set_title('Blue Channel Distribution', fontweight='bold')
    axes[1, 0].set_xlabel('Pixel Value')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].grid(True, alpha=0.3)
    
    # RGB 채널 비교
    axes[1, 1].hist(r_values, bins=30, alpha=0.5, color='red', label='Red', edgecolor='black')
    axes[1, 1].hist(g_values, bins=30, alpha=0.5, color='green', label='Green', edgecolor='black')
    axes[1, 1].hist(b_values, bins=30, alpha=0.5, color='blue', label='Blue', edgecolor='black')
    axes[1, 1].set_title('RGB Channel Comparison', fontweight='bold')
    axes[1, 1].set_xlabel('Pixel Value')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print(" 시각화 함수들 정의 완료")
# RGB 채널 분포 시각화
create_rgb_channel_plots(r_values, g_values, b_values)

# 정규화 권장사항 출력
print("=" * 60)
print(f" 정규화 권장사항:")
print("=" * 60)
print(f" 현재 ImageNet 정규화 값:")
print(f"   평균=[0.485, 0.456, 0.406], 표준편차=[0.229, 0.224, 0.225]")
print(f"Dataset 실제 평균 값:")
print(f"   평균=[{image_stats['r_channel']['mean']/255:.3f}, {image_stats['g_channel']['mean']/255:.3f}, {image_stats['b_channel']['mean']/255:.3f}]")
print(f"Dataset 실제 표준편차 값:")
print(f"   표준편차=[{image_stats['r_channel']['std']/255:.3f}, {image_stats['g_channel']['std']/255:.3f}, {image_stats['b_channel']['std']/255:.3f}]")


In [12]:
# EDA 요약 및 인사이트
print("=" * 60)
print(" EDA 요약 및 인사이트")
print("=" * 60)

print(" 주요 발견사항:")
print("1. 데이터셋 규모:")
print(f"   - 훈련 샘플: {basic_info['train_samples']:,}개")
print(f"   - 테스트 샘플: {basic_info['test_samples']:,}개")
print(f"   - 클래스 수: {basic_info['num_classes']}개")

print("\n2. 클래스 분포:")
print(f"   - 가장 많은 클래스: {class_distribution.loc[class_distribution['count'].idxmax(), 'class']} ({class_distribution['count'].max()}개)")
print(f"   - 가장 적은 클래스: {class_distribution.loc[class_distribution['count'].idxmin(), 'class']} ({class_distribution['count'].min()}개)")
print(f"   - 클래스 불균형 비율: {class_distribution['count'].max() / class_distribution['count'].min():.2f}:1")

print("\n3. 이미지 특성:")
print(f"   - 이미지 채널: {image_properties['channels']}")
print(f"   - 평균 이미지 크기: {image_properties['height_stats']['mean']:.0f} x {image_properties['width_stats']['mean']:.0f}")
print(f"   - 크기 범위: {image_properties['height_stats']['min']}-{image_properties['height_stats']['max']} x {image_properties['width_stats']['min']}-{image_properties['width_stats']['max']}")

print("\n4. 픽셀 값 분포:")
print(f"   - Red 채널 평균: {image_stats['r_channel']['mean']:.1f}")
print(f"   - Green 채널 평균: {image_stats['g_channel']['mean']:.1f}")
print(f"   - Blue 채널 평균: {image_stats['b_channel']['mean']:.1f}")

print("\n 모델링 권장사항:")
print("1. 클래스 불균형 해결:")
print("   - 클래스 가중치 적용 또는 오버샘플링 고려")
print("   - F1-score를 주요 평가 지표로 사용")

print("\n2. 이미지 전처리:")
print("   - 다양한 크기의 이미지를 고려한 augmentation 적용")
print("   - 현재 ImageNet 정규화 사용이 적절함")

print("\n3. 모델 선택:")
print("   - 작은 이미지 크기(32x32)에 적합한 모델 선택")
print("   - Transfer learning 활용 권장")

print("\n4. 하이퍼파라미터:")
print("   - 현재 설정이 적절함 (배치 크기 32, 학습률 1e-3)")
print("   - 에포크 수는 조기 종료로 조정 가능")

print("\n EDA 분석 완료!")
print("=" * 60)
